In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical

import random

In [34]:

WIDTH=150
HEIGHT=150
IMAGE_SIZE=(WIDTH, HEIGHT)
IMAGE_CHANNELS=3
batch_size = 10

In [62]:
pip install py7zr

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import py7zr

# with ZipFile('../content/drive/MyDrive/GlassMetal.zip', 'r') as zip:
#     zip.extractall()


with py7zr.SevenZipFile('../content/drive/MyDrive/UnSortedWaste.7z', mode='r') as z:
     z.extractall()


In [7]:
path =  "../content/UnSortedWaste"
train_dir = os.path.join(path, "train")
validation_dir = os.path.join(path, "validation")
filenames = os.listdir(path)

In [62]:
#delete dirs
shutil.rmtree( train_dir )
shutil.rmtree( validation_dir )



In [9]:
labels = ["cardboard", "glass", "metal", "plastic", "trash", "paper"]

os.makedirs(train_dir)
os.makedirs(validation_dir)

for label in labels:
  os.makedirs(os.path.join(train_dir, label))
  os.makedirs(os.path.join(validation_dir, label))


In [10]:
df = pd.DataFrame(data=filenames, columns=['file_name'])
total_train, total_validate = 0, 0

df['label'] = df['file_name'].apply(lambda x: "cardboard" if x[:3]=='car'
                                    else 'glass' if x[:3]=='gla'
                                    else "metal" if x[:3]=='met'
                                    else "paper" if x[:3]=='pap'
                                    else "plastic" if x[:3]=="pla"
                                    else "trash")


for label in df['label'].unique():
    files = df[df['label']==label]['file_name']
    train = np.random.choice(files, size=int(len(files)*0.8))
    test = [_ for _ in files if not (_ in train)]
    print(label, len(files), len(train), len(test))
    total_train += len(train)
    total_validate += len(test)

    for file in train:
      shutil.copyfile(os.path.join(path, file),
                     os.path.join(train_dir, label, file))

    for file in test:
      shutil.copyfile(os.path.join(path, file),
                     os.path.join(validation_dir, label, file))


glass 501 400 220
metal 410 328 189
trash 137 109 63
paper 594 475 260
cardboard 403 322 175
plastic 482 385 199


In [62]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (2,2), activation='relu', input_shape=(WIDTH, HEIGHT, IMAGE_CHANNELS)),
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(16, (2,2), activation='relu'),
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(32, (2,2), activation='relu'),

    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (2,2), activation='relu'),
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (2,2), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    #1100

    tf.keras.layers.Dense(6, activation='softmax')

])
model.summary()

In [56]:

from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='mse',
              metrics = ['accuracy'])

In [57]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( rescale = 1.0/255. ,
                                   rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )


In [58]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    target_size=IMAGE_SIZE )

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=batch_size,
                                                         class_mode  = 'categorical',
                                                         target_size = IMAGE_SIZE)

Found 1421 images belonging to 6 classes.
Found 1106 images belonging to 6 classes.


In [59]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
earlystop = EarlyStopping(patience=5,restore_best_weights=True)


In [62]:
history = model.fit(
            train_generator,
            epochs=60,
            validation_data=validation_generator,
            verbose=2,
            callbacks=[earlystop]
            )

In [62]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [62]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()


plt.show()

In [47]:

import numpy as np

from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded=files.upload()

for fn in uploaded.keys():

  # predicting images
  path='/content/' + fn
  img=load_img(path, target_size=(150, 150))

  x=img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)

  print(classes[0])

  # if classes[0]>0.5:
  #   print(fn + " is a dog")
  # else:
  #   print(fn + " is a cat")


Saving metal.jpg to metal.jpg
1/1 [==============================] - 0s 20ms/step
[4.4375195e-04 2.3015573e-07 1.1818340e-03 9.9836296e-01 7.5288435e-06
 3.8351500e-06]


In [44]:
labels.sort()
print(labels)

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
